### Reingest missing sync sessions

In [1]:
from scripts.conf_file_finding import try_find_conf_file
try_find_conf_file()

Local configuration file found !!, no need to run the configuration (unless configuration has changed)


In [2]:
import numpy as np
import u19_pipeline.ephys_pipeline as ep
import u19_pipeline.recording as recording
import datajoint as dj
import pandas as pd
import datetime


[2026-02-11 14:11:07,955][INFO]: DataJoint 0.14.4 connected to alvaros@datajoint00.pni.princeton.edu:3306


In [3]:
#(ep.BehaviorSync & "recording_id=502").delete()


### Fetch all missing sync sessions

In [4]:
past_date = datetime.date.today() - datetime.timedelta(days=2000)
query_session_date =  'session_date >="' + past_date.strftime('%Y-%m-%d') +'"'

all_recs = ((recording.Recording & "recording_modality='electrophysiology'") * (recording.Recording.BehaviorSession & query_session_date)).join(ep.BehaviorSync, left=True)
all_recs = pd.DataFrame(all_recs.fetch('recording_id','subject_fullname','session_date','session_number','sync_data',as_dict=True))

sync_recs = pd.DataFrame((recording.Recording * ep.BehaviorSync).fetch('KEY', 'regular_sync_status', 'fixed_sync_status', 'virmen_sync_status', as_dict=True))




In [8]:
def get_rec_key_dict(recording_id):

    return {'recording_id': recording_id}

In [9]:
def compare_behaviorsyncdata_values(value1, value2, keys_sync_data):

    all_equal = True
    for comp_key in keys_sync_data:
        if 'iteration_idx_vector_from_virmen' == comp_key:
            continue
        if 'iteration' in comp_key:
            if isinstance(value1[comp_key],list):
                all_equal = False
                break

            if value1[comp_key].shape == value2[comp_key].shape:
                for j in range(value1[comp_key].shape[0]):
                    if not np.array_equal(value1[comp_key][j], value2[comp_key][j]):
                        print('j', 'not equal')
                        all_equal = False
                        break

        else:
            if not np.array_equal(value1[comp_key], value2[comp_key]):
                all_equal = False
                break

    return all_equal

In [10]:
sync_recs2 = pd.merge(all_recs,sync_recs, how='left', indicator=True)
sync_recs2 = sync_recs2.loc[sync_recs2['_merge']=='both',:]

sync_recs2['regular_sync_status'] = sync_recs2['regular_sync_status'].astype(int)
sync_recs2['fixed_sync_status'] = sync_recs2['fixed_sync_status'].astype(int)
sync_recs2['virmen_sync_status'] = sync_recs2['virmen_sync_status'].astype(int)

sync_recs2 = sync_recs2.sort_values(by='session_date', ascending=False)
sync_recs2['rec_key'] = sync_recs2['recording_id'].apply(get_rec_key_dict)
sync_recs2 = sync_recs2.reset_index(drop=True)

keys_sync_data = ['trial_idx_vector', 'iteration_idx_vector', 'trial_idx_vector_from_virmen', 'iteration_idx_vector_from_virmen']

In [13]:
sync_recs2.loc[sync_recs2['sync_data'].isna(),:]

,recording_id,subject_fullname,session_date,session_number,sync_data,regular_sync_status,fixed_sync_status,virmen_sync_status,_merge,rec_key
127,484,jyanar_ya032,2025-08-05,0,None,0,0,0,both,{'recording_id': 484}
501,84,ms81_M012,2021-12-11,0,None,0,0,0,both,{'recording_id': 84}
502,41,ms81_M012,2021-12-10,0,None,0,0,0,both,{'recording_id': 41}
509,87,ms81_M011,2021-11-18,0,None,0,0,0,both,{'recording_id': 87}
512,35,ms81_M004,2021-05-08,0,None,0,0,0,both,{'recording_id': 35}
513,37,ms81_M005,2021-05-08,0,None,0,0,0,both,{'recording_id': 37}
514,31,ms81_M003,2021-05-07,0,None,0,0,0,both,{'recording_id': 31}
515,34,ms81_M004,2021-05-07,0,None,0,0,0,both,{'recording_id': 34}
516,33,ms81_M004,2021-05-06,0,None,0,0,0,both,{'recording_id': 33}
517,30,ms81_M005,2021-05-06,0,None,0,0,0,both,{'recording_id': 30}


### Execute sync code

In [ ]:
list_missing_new_sync = []
list_already_synced = []
for i in range(sync_recs2.shape[0]):

    if Cis not None and isinstance(sync_recs2.loc[i,'sync_data']['iteration_idx_vector_from_virmen'], list):
            list_missing_new_sync.append(sync_recs2.loc[i, 'rec_key'])

            final_key = ep.BehaviorSync().make(sync_recs2.loc[i,'rec_key'], **{'populate': False})

            if final_key is not None:
                all_equal = compare_behaviorsyncdata_values(sync_recs2.loc[i,'sync_data'],final_key['sync_data'],keys_sync_data)

            if final_key is not None:
                update_key = sync_recs2.loc[i, 'rec_key'].copy()
                update_key['sync_data'] = final_key['sync_data'].copy()
                ep.BehaviorSync.update1(update_key)
    else:
         list_already_synced.append(sync_recs2.loc[i, 'rec_key'])
    


In [ ]:
not_synced_at

[{'recording_id': 616},
 {'recording_id': 613},
 {'recording_id': 615},
 {'recording_id': 612},
 {'recording_id': 614},
 {'recording_id': 610},
 {'recording_id': 608},
 {'recording_id': 609},
 {'recording_id': 607},
 {'recording_id': 606},
 {'recording_id': 602},
 {'recording_id': 600},
 {'recording_id': 599},
 {'recording_id': 597},
 {'recording_id': 598},
 {'recording_id': 596},
 {'recording_id': 605},
 {'recording_id': 595},
 {'recording_id': 594},
 {'recording_id': 593},
 {'recording_id': 592},
 {'recording_id': 591},
 {'recording_id': 604},
 {'recording_id': 589},
 {'recording_id': 587},
 {'recording_id': 585},
 {'recording_id': 586},
 {'recording_id': 588},
 {'recording_id': 584},
 {'recording_id': 603},
 {'recording_id': 583},
 {'recording_id': 581},
 {'recording_id': 580},
 {'recording_id': 578},
 {'recording_id': 577},
 {'recording_id': 576},
 {'recording_id': 575},
 {'recording_id': 571},
 {'recording_id': 570},
 {'recording_id': 569},
 {'recording_id': 567},
 {'recording_id'

In [5]:
final_key =ep.BehaviorSync().make({'recording_id': 484}, **{'populate': True})


if final_key is not None:
    update_key = {'recording_id': 484}.copy()
    update_key['sync_data'] = final_key['sync_data'].copy()
    #ep.BehaviorSync.update1(update_key)

{'recording_id': 484}
get_ephys_root_data_dir ['/mnt/cup/braininit/Data/Raw/electrophysiology', '/mnt/cup/braininit/Data/Processed/electrophysiology']
get_session_directory(recording_key) jyanar/jyanar_ya032/20250805_g0/ya032_20250805_g0
ephys dir: /mnt/cup/braininit/Data/Raw/electrophysiology/jyanar/jyanar_ya032/20250805_g0/ya032_20250805_g0
{'subject_fullname': 'jyanar_ya032', 'session_date': datetime.date(2025, 8, 5), 'session_number': 0}
len iterstart 241
after reading behavior data
['0', '7']
nChan: 1, nFileSamp: 17948970
after reading spikeglx data
len trial_start_idx (241,)
mode deduction:  counter_bit0
metrics to evaluate...
0 [] [] 241
Synced perfectly xxxxxxxxxxxxxx
after all main ehpys fix sync code 1 0
ephys_session_fullpath /mnt/cup/braininit/Data/Raw/electrophysiology/jyanar/jyanar_ya032/20250805_g0/ya032_20250805_g0/ya032_20250805_g0_t0.nidq.bin
sync code executed sucessfully !!!!!!!!!!!!!!!!!!!!!!!!!!!
{'populate': True}
No imec meta file found.


In [16]:
(ep.BehaviorSync & {'recording_id': 484}).delete()

[2026-02-11 13:52:55,065][INFO]: Deleting 1 rows from `u19_ephys_pipeline`.`_behavior_sync__imec_sampling_rate`
[2026-02-11 13:52:55,071][INFO]: Deleting 1 rows from `u19_ephys_pipeline`.`_behavior_sync`
[2026-02-11 13:53:00,327][INFO]: Deletes committed.


1

In [19]:
final_key

{'recording_id': 464,
 'nidq_sampling_rate': 5000.062709,
 'iteration_index_nidq': [nan],
 'trial_index_nidq': [nan],
 'sync_data': {'trial_idx_vector': array([ 239760,  295292,  445123,  578687,  661122,  739833,  817214,
          859996,  901828,  944523,  986587, 1029567, 1103112, 1184894,
         1232134, 1266000, 1302690, 1339846, 1392071, 1432904, 1474234,
         1528057, 1569038, 1612483, 1709161, 1754980, 1796851, 1841596,
         1876319, 1919184, 1960500]),
  'iteration_idx_vector': array([array([239760, 239911, 240007, 240206, 240340, 240430, 240505, 240563,
                240617, 240673, 240731, 240785, 240837, 240891, 240945, 240998,
                241050, 241109, 241172, 241223, 241277, 241333, 241386, 241441,
                241494, 241546, 241601, 241655, 241708, 241763, 241818, 241871,
                241923, 241979, 242030, 242083, 242136, 242189, 242242, 242295,
                242349, 242401, 242453, 242507, 242559, 242611, 242665, 242717,
                242